# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from g_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
P1_df = pd.read_csv("C:/Users/19739/Desktop/New folder/Project_4/Resources/FD_List.csv")
P1_df

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#humidity = P1_df["Humidity"].astype(float)
#maxhumidity = humidity.max()
locations = P1_df[["Latitude", "Longitude"]]

In [5]:
fig = gmaps.figure()
#heat_layer = gmaps.heatmap_layer(locations, weights=tate,dissipating=False, max_intensity=maxhumidity,point_radius=3)
#fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:

criteria_city_df = P1_df.loc[(P1_df["Max Temp"] > 70) & (P1_df["Max Temp"] < 80) & (P1_df["Cloudiness"] == 0) & (P1_df["Wind Speed"] <= 10), :]
criteria_city_df = criteria_city_df.dropna(how='any')
criteria_city_df.reset_index(inplace=True)
del criteria_city_df['index']
criteria_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Grasse,43.6667,6.9167,73.26,59,0,4.61,FR,1631208561
1,Temba,-25.3803,28.2631,71.10,28,0,8.99,ZA,1631208593
2,Cape Town,-33.9258,18.4232,72.10,52,0,1.99,ZA,1631208439
3,Wencheng,33.0559,113.7978,71.94,89,0,5.41,CN,1631208601
4,Ürümqi,43.8010,87.6005,70.75,33,0,6.71,CN,1631208607
5,Dimona,31.0667,35.0333,79.84,35,0,5.57,IL,1631208634
6,Villarrica,-25.7500,-56.4333,74.07,30,0,8.03,PY,1631208638
7,Tongchuanshi,35.0806,109.0897,70.52,62,0,2.68,CN,1631208665
8,Tūkrah,32.5341,20.5791,76.78,59,0,7.76,LY,1631208679


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:

hotel_df = pd.DataFrame(criteria_city_df, columns=["City", "Country", "Lat", "Lng"])

hotel_df

,City,Country,Lat,Lng
0,Grasse,FR,43.6667,6.9167
1,Temba,ZA,-25.3803,28.2631
2,Cape Town,ZA,-33.9258,18.4232
3,Wencheng,CN,33.0559,113.7978
4,Ürümqi,CN,43.8010,87.6005
5,Dimona,IL,31.0667,35.0333
6,Villarrica,PY,-25.7500,-56.4333
7,Tongchuanshi,CN,35.0806,109.0897
8,Tūkrah,LY,32.5341,20.5791


In [ ]:

# FD variable
FD = []

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(P1_df["City"])):

    lat = P1_df.loc[city]["Latitude"]
    lng = P1_df.loc[city]["Longitude"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "Fire Department",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    FD_request = requests.get(base_url, params=params)
    FD_response = FD_request.json()

    try:
        FD.append(FD_response["results"][0]["name"])
    except:
        FD.append("Nearest FD not found")

# Dataframe with nearest hotel
FD_df["FD Name"] = FD
FD_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
FD_info = [info_box_template.format(**row) for index, row in P1_df.iterrows()]
locations = P1_df[["Latitude", "Longitude"]]

In [10]:

# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig


Figure(layout=FigureLayout(height='420px'))